<a href="https://colab.research.google.com/github/cc4351/teamtracking/blob/master/1122_Chen_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
import statistics
import math
from time import time
import os
from collections import defaultdict
import pickle
import plotly.express as px
import queue
import pickle
import sys

In [2]:
# replace the root path
# switching into the folder containing all the data and listdir
%cd '/content/drive/My Drive/Colab Notebooks/Molecule/data/RNN_train' 
!ls

/content/drive/My Drive/Colab Notebooks/Molecule/data/RNN_train
model.png		 receptorInfoLabeledPB.mat  sim4
receptorInfoAll.mat	 samplePBnnMatrix.pkl	    tracksFinal.mat
receptorInfoLabeled.mat  sim3


# Notes on variables
- movieInfo: output generated from u-track program with the simulated track movies from FISIK
    - shape: (# of frames, 1), one row per entry, eg (4000, 1) for sim4 MovieInfo
    - headers for each entry: 'xCoord', 'yCoord', 'amp', 'sigma', 'pValue', 'resnorm'
    - To get access to the xCoord information of the Yth particle on frame X would require -> movieInfo[X]['xCoord'][0][Y]
- groundTruth: real track trajectories generated by FISIK
    - shape: (# of tracks, 2 dimensional data point, # of frames), eg (53, 2, 4001) for sim4
    - To get access to the x, y coordinate information of the Yth particle on frame X would require -> groundTruth[X, :, Y]




# RNN pipeline

## data pre-processing

In [3]:
rootPath = r'sim4'
movieInfo = loadmat(os.path.join(rootPath, 'movieInfo.mat'))['movieInfo'] # movieInfo by utrack
groundTruth = loadmat(os.path.join(rootPath, 'update_groundTruth.mat'))['groundTruth'] # ground truth by FISIK
mask = loadmat(os.path.join(rootPath, 'mask2mat.mat'))['kk'] # cell bound mask

In [4]:
# obtain the masked groundTruth, aka get rid of the out of bound particles
def mask_ground_truth(groundTruth, mask):
    masked_ground_truth = np.copy(groundTruth) # deep copy groundTruth
    x_width = len(mask)
    if x_width == 0:
        print("empty mask")
        return
    y_width = len(mask[0])
    if y_width == 0:
        print("empty mask")
        return
    for id1, traj in enumerate(groundTruth):
        # acknowledge the particle if its nearest grid is masked as 1
        for id2 in range(traj.shape[1]):
            ptr = traj[:,id2]
            # leave nan value as is
            if np.isfinite(ptr[0]) and np.isfinite(ptr[1]):
                nearest_x = int(round(ptr[0]))
                nearest_y = int(round(ptr[1]))
                if nearest_x >= x_width or nearest_y >= y_width or mask[nearest_x][nearest_y] == 0:
                    masked_ground_truth[id1,:,id2] = -1.0
                    # print(f'x {ptr[0]}, y {ptr[1]} is out of bound at id1 {id1} id2 {id2}')
    return masked_ground_truth
        

In [5]:
# extract the xy coordinates 
def movieinfo_to_particle_matrix(movieInfo):
    particle_matrix = []
    for frame in movieInfo:
        if len(frame['xCoord'][0]) == 0:
            assert len(frame['yCoord'][0]) == 0
            particle_matrix.append(np.asarray([]))
            continue
        xCoord = frame['xCoord'][0][:,0]
        yCoord = frame['yCoord'][0][:,0]
        assert len(xCoord) == len(yCoord)
        xy = np.asarray([[xCoord[i], yCoord[i]] for i in range(len(xCoord))])
        particle_matrix.append(xy)
    assert len(particle_matrix) == len(movieInfo)
    print(f"== successfully converted movieInfo into particle matrix with {len(movieInfo)} frames")
    return particle_matrix

In [6]:
masked_groundTruth = mask_ground_truth(groundTruth, mask)
particle_matrix = movieinfo_to_particle_matrix(movieInfo)

== successfully converted movieInfo into particle matrix with 4000 frames


## Analysis of Simulated Data

In [ ]:
# confirming that the minimal distance is small and that the coordinates match
numFrames = len(movieInfo)
catFrame = [np.vstack((movieInfo['xCoord'][i][0][:,0], movieInfo['yCoord'][i][0][:,0])).T 
            if len(movieInfo['xCoord'][i][0])>0 else np.asarray([])for i in range(numFrames)]
for x, y in catFrame[0]:
    min_dis = sys.maxsize
    min_a = -1
    min_b = -1
    for a, b in groundTruth[:,:,0]:
        dist = (x-a)**2 + (b-y)**2
        if dist < min_dis:
            min_dis = dist
            min_a = a
            min_b = b
    print(x, y, min_a, min_b, min_dis)

In [ ]:
# visualize the first track
track1 = groundTruth[0,:,:]
df = pd.DataFrame(track1.transpose(), columns=['x', 'y'])
px.scatter_3d(df, x='x', y='y', z=df.index, size=np.ones(df.shape[0])*1)

In [8]:
# helper function to find the nearest point in arr to pt and their L2 distance
def nearest_pt(arr, pt):
    arr = np.asarray(arr)
    idx = np.argmin(np.linalg.norm(arr-pt, axis=1))
    return arr[idx], np.linalg.norm(arr[idx]-pt)

'''
Function: count the number of particles that are
    - generated by ground truth && in range, and is identified by utrack (true positive)
    - generated by ground truth && in range, and not identified by utrack (false negative)
    - not generated by ground truth, and is falsely identified by utrack (false positive)
Input:
    - masked_ground_truth: ground truth data masked with the cell bound mask
        could be obtained by using mask_ground_truth(groundTruth, mask)
    - particleMatrix: extracted xy coordinates from utrack output movieInfo
        could be obtained by using movieinfo_to_particle_matrix(movieInfo)
    - search_range: the margin of error allowed for utrack program
        in utrack it searches for particles within 3 pixels' distance
        by default, we will identify the particle as being correctly identified
        by utrack if there exists a xy pair (x_utrack, y_utrack) for a given
        ground truth pair such that 
        (x_truth-x_utrack)^2 + (y_truth-y_utrack)^2 <= 4 (aka within 2 pixels)
Output:
    - hit miss counts
'''
# masked_ground_truth: (# of track, 2, # of frames)
# particleMatrix: (# of frames, [# of particles, 2])
def count_hit_rate_distance(masked_ground_truth, particleMatrix, search_range = 2):
    true_positive = 0
    false_negative = 0
    false_positive = 0
    total_truth_count = 0
    total_predict_count = 0
    # compare on a per frame basis
    lenTruth = masked_ground_truth.shape[2]
    lenUtrack = len(particleMatrix)
    print(f'ground truth has {lenTruth}, particleMatrix has {lenUtrack} frames respectively')
    assert lenTruth >= lenUtrack # makes sure that the # of truth frames >= # of utrack frames
    num_of_frame = min(lenTruth, lenUtrack)
    for frame_number in range(num_of_frame):
        valid_arr = []
        for pt in masked_ground_truth[:,:,frame_number]:
            if pt[0] == -1 or pt[1] == -1 or np.isfinite(pt[0]) == False or np.isfinite(pt[1]) == False:
                continue
            valid_arr.append(pt)
            total_truth_count += 1
            if len(particleMatrix[frame_number]) == 0:
                false_negative += 1
                continue
            _, dist = nearest_pt(particleMatrix[frame_number], pt)
            # true positive
            if dist <= search_range:
                true_positive += 1
            else: # false negative
                false_negative += 1
        for pt in particleMatrix[frame_number]:
            _, dist = nearest_pt(valid_arr, pt)
            total_predict_count += 1
            # false positive
            if dist > search_range:
                false_positive += 1
    return dict({'tp':true_positive, 'fn':false_negative, 'fp':false_positive, 
                'ttc':total_truth_count, 'tpc':total_predict_count})


In [ ]:
hit_dist_output = count_hit_rate_distance(masked_groundTruth, particle_matrix)
hit_dist_output

ground truth has 4001, particleMatrix has 4000 frames respectively


{'fn': 11162, 'fp': 8934, 'tp': 42810, 'tpc': 51704, 'ttc': 53972}

In [ ]:
# on particle level, by arbitrary search_range
tp = float(hit_dist_output['tp'])
fp = float(hit_dist_output['fp'])
fn = float(hit_dist_output['fn'])
hit_precision = tp/(tp+fp)
hit_recall = tp/(tp+fn)
acc = 2*hit_precision*hit_recall/(hit_precision + hit_recall)
print(f'by distance: acc {acc}, hit_precision {hit_precision}, hit_recall {hit_recall}')

by distance: acc 0.8099057853115895, hit_precision 0.8273423005565863, hit_recall 0.7931890609945897


In [ ]:
# TODO: Instead of using an arbitrary search_range,
# we can also choose to use the std of the coordinates that
# we have obtained from utrack to as the search_range
# for example, if the true pt and the predict pt is 1 std away,
# we consider it to be a true positive
def count_hit_rate_std():
    pass

## Prepare Inputs to Model

In [13]:
'''
Function: nnMatrix is used to find the nearest neighbors for all particles
    e.g. For every particle pt in frame t in the particle matrix <particles>,
    we return its <num_neighbors> nearest neighbors in frame t+1, as potential
    candidates for linking. Note that we also pad the nearest_neighbors output
    with an additonal -1, which could be used to indicate track termination and
    should always be considered likely by the neural network.
Input:
    num_neighbors: the number of non-termination nearest neighbors to consider
    particles: particle matrix, should be obtained by running movieinfo_to_particle_matrix(movieInfo)
        using the movieInfo struct obtained from utrack
    
'''
def nnMatrix(num_neighbors, particles):
    particleInfo = particles
    loc_dic = defaultdict(dict)
    idx = 0
    for item in enumerate(particleInfo[:-1]):
        idx = item[0]
        frame = item[1]
        nextFrame = particleInfo[idx+1]
        # if frame is empty, insert an empty dictionary
        if len(frame) == 0:
            loc_dic[idx] = {}
        for ptr in enumerate(frame):
            pt = ptr[1]
            if any(np.isnan(pt)) == True:
                continue
            dist = [np.sum((pt-p2)**2) for p2 in nextFrame]
            dist = np.asarray(dist)
            i = np.argsort(dist) # returns the indices of dist sorted by the value/distances from pt
            ret = i[:num_neighbors] # get indices for the first <num_neighbors> particles sorted by distance
            ret = np.append(ret, [-1])
            loc_dic[idx][ptr[0]] = {'x': pt[0], 'y': pt[1], 'nbrs': ret}
    # add x-y coordinate for last frame
    for ptr in enumerate(particleInfo[-1]):
        if np.isfinite(ptr[1][0]):
            loc_dic[idx+1][ptr[0]] = {'x':ptr[1][0], 'y':ptr[1][1], 'nbrs':[-1]}
    return list(loc_dic.values())

In [14]:
nn_matrix = nnMatrix(3, particle_matrix)

In [25]:
'''
getHypTracks: get possible hypotheses padded with [-1,-1] termination
Input
    nnMatrics: nn_matrix obtained by running nnMatrix() function on particle_matrix
    startFrame: frame index of the start particle
    startIdx: the idx/array index of the start particle in that frame
    numNN: the number of nearest neighbors, note that this value should be matching
        with the num_nbrs value passed into nnMatrix() function
    forwardProp: the number of frames we forecast into, set to 3 by default
Output
    a list of numpy arrays represents possible tracks start from start particle
    and ends in the last frame we forecast into, all in xy coordinates form
'''
def getHypTracks(nnMatrices, startFrame, startIdx, numNN, forwardProp = 3):
    # numTrack: total number of track under consideration
    numTrack = int((numNN**(forwardProp+1)-1)/(numNN-1))
    numFrames = len(nnMatrices) # number of frames
    fakeDict = {'x':-1, 'y':-1, 'nbrs':[]} # dummy dictionary in case empty
    padding = [-1, -1] # dummy xy coordinates to signal termination
    '''
    helper: utility function to do a binary tree traversal and build tracks
    Input:
        frame: the frame index of the start particle
        idx: the idx/array index of the start particle in that frame
        depth: forwardPropragation level
    '''
    def helper(frame, idx, depth):
        if frame >= numFrames or depth > forwardProp or (idx > -1 and idx not in nnMatrices[frame]):
            return []
        # idx == -1 signals termination, use the dummy dictionary
        if idx == -1:
            root = fakeDict
        # else, query the nnMatrices dictionary to get the xy coordinates
        else:
            root = nnMatrices[frame][idx]
        xy = [root['x'], root['y']]
        # recursively call the helper function to generate all tracks till termination
        results = [xy+path for kid in root['nbrs'] if kid != None for path 
                   in helper(frame+1, kid, depth+1)] or [xy]
        return results
    results = []
    for track in helper(startFrame, startIdx, 0):
        endPadding = forwardProp + 1 - int(len(track)/2)
        tmp = padding + track + padding*(1+endPadding)
        tmp = np.asarray(tmp).reshape(-1, 2)
        results.append(tmp)
    if len(results) == 0:
        print(f"no {startIdx} in {startFrame}")
        return results
    fakeTrack = np.asarray([padding for i in range(forwardProp+3)])
    for _ in range(numTrack - len(results)):
        results.append(fakeTrack)
    return results

In [24]:
# ref: https://stackoverflow.com/a/16193445
'''
getTruth: generate the ground truth tracks to compare against the predicte tracks
    from the RNN model. Note that this function makes the following assumptions
        - the true particle is the closest in euclidean distance with the predicted
          particle, aka we assume that utrack particle identification is mostly correct
        - consistent with the hit/miss ratio analysis, we mark a track as nonexistent
          if there does not exist a "real" particle in the groundTruth in that frame
          that is the within 2 pixel radius of the predicted particle by default, note that
          the search_range can also be customized

Input:
    groundTruth: the groundTruth matrix obtained from the groundTruth.mat matlab matrix
    frameIdx: the frame the predict particle is in
    predict: the predicted particle from utrack, should be a list [predict_x, predict_y]
        storing its xy coordinate information
    search_range: the effective radius to determine whether the predict particle matches 
        its closest neighbor in truth, default to 2
Output:
    truth_track: the index of the real trajectory
    exist: if the track should exist, boolean value
    
'''    
# get ground truth labels

def getTruth(groundTruth, frameIdx, predict, search_range = 2):
    # helper function to find the nearest point in arr to pt and their L2 distance
    def nearest_truth(arr, pt):
        arr = np.asarray(arr)
        idx = np.argmin(np.linalg.norm(arr-pt, axis=1))
        return idx, np.linalg.norm(arr[idx]-pt)

    frame = groundTruth[:,:,frameIdx]
    truth_idx, dist = nearest_truth(frame, predict)
    # if out of effective range, return false directly
    if dist > search_range:
        return [], False
    else:
        return truth_idx, True

In [ ]:
d = 2
numNN = 3
batchNumTrack = int((numNN**(d+1)-1)/(numNN-1))
print("batchNumTrack: ", batchNumTrack)
allX = []
allAsgn = []
allExt = []

numFrames = len(nn_matrix) -1

for t in range(numFrames): # frame number
    for k, v in nn_matrix[t].items():
        tracks = getHypTracks(nn_matrix, t, k, numNN, d)
        assert len(tracks) == batchNumTrack or len(tracks) == 0
        if len(tracks) == 0:
            print(f"empty track for frame {t} and id {k}")
            break
        allX.append(tracks)
        truth, existence = getTruth(groundTruth, t, [v['x'], v['y']])
        # TODO: finishing here

In [30]:
for k, v in nn_matrix[-1].items():
    print(k, v)
# nn_matrix[-1]

0 {'x': 93.46928848596423, 'y': 165.68498200875104, 'nbrs': [-1]}
1 {'x': 116.24390638891667, 'y': 124.82137459859109, 'nbrs': [-1]}
2 {'x': 157.64123627793668, 'y': 25.99502861721942, 'nbrs': [-1]}
3 {'x': 159.98154699495583, 'y': 51.63154936249261, 'nbrs': [-1]}


In [ ]:
# unit test before training
# the index thingy need to be randomized/sorted in the u-track way

len(allX), len(allAsgn), len(allExt)

(50840, 50840, 50840)

In [ ]:
# numNN = 3
# d = 2
# t = 4000
# idx = 7
# # nnMatrices = nnMatrix(3, movieInfo)
# tracks = getHypTracks(nnMatrices, t, idx, numNN, d)
# # remember to check if t is the last frame
# truth, existence = getTruth(labeledTracks[idx,:,t:t+2].T, nnMatrices[t], idx, t, numNN)
# truth, existence
import random

c = list(zip(allX, allAsgn, allExt))
random.shuffle(c)
allX, allAsgn, allExt = zip(*c)
allX = list(allX)
allAsgn = list(allAsgn)
allExt = list(allExt)

allAsgn[0], allExt[0]

([1, 0, 0, 0], [1, 0])

In [ ]:
allX[0]

[array([[-1.        , -1.        ],
        [ 6.01981178, 12.47991293],
        [ 5.94978352, 12.4493243 ],
        [ 5.93745409, 12.49473248],
        [-1.        , -1.        ]]), array([[-1.        , -1.        ],
        [ 6.01981178, 12.47991293],
        [ 5.94978352, 12.4493243 ],
        [ 6.30155956, 15.9512826 ],
        [-1.        , -1.        ]]), array([[-1.        , -1.        ],
        [ 6.01981178, 12.47991293],
        [ 5.94978352, 12.4493243 ],
        [ 2.77884018,  9.43847762],
        [-1.        , -1.        ]]), array([[-1.        , -1.        ],
        [ 6.01981178, 12.47991293],
        [ 5.94978352, 12.4493243 ],
        [-1.        , -1.        ],
        [-1.        , -1.        ]]), array([[-1.        , -1.        ],
        [ 6.01981178, 12.47991293],
        [ 6.35341208, 15.98680597],
        [ 6.30155956, 15.9512826 ],
        [-1.        , -1.        ]]), array([[-1.        , -1.        ],
        [ 6.01981178, 12.47991293],
        [ 6.35341208, 1

## model

In [ ]:
# ref: https://keras.io/api/models/model_training_apis/
# model itself
class rnnModel(keras.Model):
    def __init__(self, numNN, t, d, numFilter = 12, k=64, numDim = 2, dropout = 0.25, t_init = 0, batch_size = None):
        super(rnnModel, self).__init__()
        dt = t + d + 1 - t_init 
        rate = dropout
        # (numNN+1)**d := total number of hypotheses to consider
        # dt := length of each track
        # numDim := length of each datapoint 
        inputDimOne = int((numNN**(d+1)-1)/(numNN-1))
        input_size = (inputDimOne, dt, numDim)
        ins = keras.Input(shape=input_size, batch_size=batch_size)
        x = layers.Conv1D(numFilter, 3 , activation='relu',input_shape=input_size[2:])(ins)
        x = tf.reshape(x, x.shape[1:])
        x = layers.Bidirectional(layers.LSTM(k, return_sequences=True))(x)
        x = layers.Bidirectional(layers.LSTM(k, return_sequences=True))(x)
        x = layers.Bidirectional(layers.LSTM(k))(x)
        x = layers.Dense(k, activation='relu')(x)
        x = layers.GaussianDropout(rate)(x)
        x = layers.Dense(1, activation='relu')(x)
        x = layers.GaussianDropout(rate)(x)
        x = tf.reshape(x, [1, x.shape[0], x.shape[1]])
        x = layers.MaxPool1D(pool_size=(numNN+1)**(d-1), name="pool1", padding="same")(x)
        x = tf.reshape(x, [x.shape[0], x.shape[1]])
        z = layers.Dense(k, input_shape =(numNN+1,1), activation='relu')(x)
        z = layers.GaussianDropout(rate)(z)
        z = layers.Dense(k, activation='relu')(z)
        z = layers.GaussianDropout(rate)(z)
        z = layers.Dense(numNN+1, activation='relu')(z)
        f1_output = layers.Softmax(name="assignment")(z)

        y = layers.Dense(k, input_shape =(numNN+1,1), activation='relu')(x)
        y = layers.GaussianDropout(rate)(y)
        y = layers.Dense(k, activation='relu')(y)
        y = layers.GaussianDropout(rate)(y)
        y = layers.Dense(2, activation='relu')(y)
        f2_output = layers.Softmax(name="existence")(y)

        ensemble = keras.Model(inputs=ins, outputs=[f1_output, f2_output])

        opt = keras.optimizers.Adam(
            learning_rate=0.001,
            beta_1=0.9,
            beta_2=0.999,
            epsilon=1e-07,
            amsgrad=False,
            name="Adam",
        )

        losses = {
            "assignment": "categorical_crossentropy",
            "existence": "categorical_crossentropy",
        }

        lossWeights = {"assignment": 1.0, "existence": 1.0}
        # keras.utils.plot_model(ensemble, show_shapes=True)
        ensemble.compile(
            optimizer=opt,
            loss=losses,
            metrics=['accuracy'],
            loss_weights=lossWeights,
            weighted_metrics=None,
            run_eagerly=None,
        )

        self.model = ensemble
    def call(self, tracks, batch_size = 1, training=False):
        return self.model.predict(tf.constant(tracks), batch_size = batch_size)
        
    def train(self, tracks, labels, epochs = 10, batch_size = 1, callbacks=[], verbose = 1):
        defaultCallback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
        return self.model.fit(tf.constant(tracks), labels,
                    epochs=epochs, batch_size=batch_size, callbacks=[defaultCallback],verbose=verbose)

In [ ]:
d = 2
numNN = 3
sampleModel = rnnModel(numNN = numNN, t = 2, d = d)

In [ ]:
# tfX = []
# for X in allX:
#     tfX.append(tf.constant(X))
dumX = tf.constant(allX)
asg = tf.constant(allAsgn)
ext = tf.constant(allExt)
dumX.shape, asg.shape, ext.shape

(TensorShape([50840, 13, 5, 2]),
 TensorShape([50840, 4]),
 TensorShape([50840, 2]))

In [ ]:
history = sampleModel.train(dumX, [asg, ext])

Epoch 1/10
50840/50840 [==============================] - 564s 11ms/step - loss: 0.0435 - assignment_loss: 0.0435 - existence_loss: 3.0893e-08 - assignment_accuracy: 0.9959 - existence_accuracy: 1.0000
Epoch 2/10
50840/50840 [==============================] - 543s 11ms/step - loss: 0.0353 - assignment_loss: 0.0353 - existence_loss: 2.8138e-11 - assignment_accuracy: 0.9959 - existence_accuracy: 1.0000
Epoch 3/10
50840/50840 [==============================] - 547s 11ms/step - loss: 0.0343 - assignment_loss: 0.0343 - existence_loss: 9.3792e-12 - assignment_accuracy: 0.9959 - existence_accuracy: 1.0000
Epoch 4/10
50840/50840 [==============================] - 577s 11ms/step - loss: 0.0333 - assignment_loss: 0.0333 - existence_loss: 3.2827e-11 - assignment_accuracy: 0.9959 - existence_accuracy: 1.0000
Epoch 5/10
50840/50840 [==============================] - 564s 11ms/step - loss: 0.0328 - assignment_loss: 0.0328 - existence_loss: 7.0344e-12 - assignment_accuracy: 0.9959 - existence_accurac

In [ ]:
# TO-DO
# 1. batch size
# 2. lr_finder
# 3. think about the track prcessing by 13 vs. by 1, 
#    and the negative effect of padding the input with [-1]*hypTrackLength (aka that's what I do rn)
# 4. track maintenance